<a href="https://colab.research.google.com/github/abhishekrathoreiitb/maven-project/blob/master/gliner_poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
!{sys.executable} -m pip install geopy gliner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.6 MB/s eta 0:00:00


In [ ]:
from gliner import GLiNER

# Initialize GLiNER with the base model
model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/781M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/781M [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

gliner_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
import re
# Labels for entity prediction
labels = ["City", "State", "Country"]

# Define the process_message function
def process_message(message):
    # Extract the address from Field 50K
    def extract_address(message):
        address_match = re.search(r":50K:.*?\n(.*?)\n(.*?)\n(.*?)\n(.*?)\n(.*?)\n", message, re.DOTALL)
        if address_match:
            return "\n".join(address_match.groups())
        return ""

    # Extract the address field
    address = extract_address(message)

    # Use the extracted address as the text for entity prediction
    text = address

    # Perform entity prediction
    entities = model.predict_entities(text, labels, threshold=0.5)

    # Filter to use only the last "City"
    city = None
    state = None
    country = None
    for entity in entities:
        if entity["label"] == "City":
            city = entity["text"]
        elif entity["label"] == "State":
            state = entity["text"]
        elif entity["label"] == "Country":
            country = entity["text"]

    # Display the preferred city, state, and country
    if city:
        print(city, "=> City")
    if state:
        print(state, "=> State")
    if country:
        print(country, "=> Country")

# Example SWIFT MT103 message
message = """
:50K:/12345678
2 Changi Business Park Cres
Singapore 486029
:52A:DEUTDEFFXXX
:53B:/DE12345678901234567890
:54A:CHASUS33XXX
:56C:IRVTUS3NXXX
:57A:NORDDKKKXXX
:59:/DK5000400440116243
JANE SMITH
789, REAL ROAD
REALVILLE
"""

# Process the example message
process_message(message)

Singapore => Country


In [ ]:
# Example SWIFT MT103 message
message = """
:50K:/12345678
Apple Inc.
1 Apple Park Way
Cupertino, California, United States
:52A:DEUTDEFFXXX
:53B:/DE12345678901234567890
:54A:CHASUS33XXX
:56C:IRVTUS3NXXX
:57A:NORDDKKKXXX
:59:/DK5000400440116243
JANE SMITH
789, REAL ROAD
REALVILLE
"""

# Process the example message
process_message(message)

Cupertino => City
California => State
United States => Country


In [ ]:
# Example SWIFT MT103 message
message = """
:50K:/12345678
Joen Doe
95/259
GandhiNagar Mandia Road
Pali Rajasthan India
:52A:DEUTDEFFXXX
:53B:/DE12345678901234567890
:54A:CHASUS33XXX
:56C:IRVTUS3NXXX
:57A:NORDDKKKXXX
:59:/DK5000400440116243
JANE SMITH
789, REAL ROAD
REALVILLE
"""

# Process the example message
process_message(message)

Pali => City
Rajasthan => State
India => Country


In [ ]:
# Example SWIFT MT103 message
message = """
:50K:/12345678
Joen Doe
AMAZON BUILDING, Road No. 2,
Financial District,
Nanakramguda Hyderabad
Telangana India 500032
:52A:DEUTDEFFXXX
:53B:/DE12345678901234567890
:54A:CHASUS33XXX
:56C:IRVTUS3NXXX
:57A:NORDDKKKXXX
:59:/DK5000400440116243
JANE SMITH
789, REAL ROAD
REALVILLE
"""

# Process the example message
process_message(message)

Hyderabad => City
Telangana => State
India => Country


In [ ]:
# Example SWIFT MT103 message
message = """
:50K:/12345678
Joen Doe
5 Ajmer Road
New Delhi Gate
Jaipur Rajasthan India
:52A:DEUTDEFFXXX
:53B:/DE12345678901234567890
:54A:CHASUS33XXX
:56C:IRVTUS3NXXX
:57A:NORDDKKKXXX
:59:/DK5000400440116243
JANE SMITH
789, REAL ROAD
REALVILLE
"""

# Process the example message
process_message(message)

Jaipur => City
Rajasthan => State
India => Country
